# Part 3. Model Training & Evaluation - biLSTM

## Prepare embedding matrix and metadata

In [1]:
import json
from pathlib import Path

import numpy as np

embedding_path = Path("models/embedding_matrix_oov.npy")
index_from_word_path = Path("models/index_from_word_oov.json")

embedding_matrix = np.load(embedding_path)
with index_from_word_path.open() as f:
    index_from_word = json.load(f)

    

## Prepare dataset

In [2]:
from utils.text import tokenize
from datasets import load_dataset

dataset = load_dataset("rotten_tomatoes")
train_dataset = tokenize(dataset["train"])
val_dataset = tokenize(dataset["validation"])
test_dataset = tokenize(dataset["test"])

/Users/bern/anaconda3/envs/nlp_project_new/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /Users/bern/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package treebank to /Users/bern/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package punkt_tab to /Users/bern/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [3]:
from utils.text import token_to_index

train_dataset = token_to_index(dataset=train_dataset, index_from_word=index_from_word)
val_dataset = token_to_index(dataset=val_dataset, index_from_word=index_from_word)
test_dataset = token_to_index(dataset=test_dataset, index_from_word=index_from_word)

In [4]:
train_dataset = train_dataset.select_columns(["label", "original_len", "indexes"])
val_dataset = val_dataset.select_columns(["label", "original_len", "indexes"])
test_dataset = test_dataset.select_columns(["label", "original_len", "indexes"])

In [5]:
train_dataset.set_format(type="torch")
val_dataset.set_format(type="torch")
test_dataset.set_format(type="torch")

In [6]:
train_dataset

Dataset({
    features: ['label', 'original_len', 'indexes'],
    num_rows: 8530
})

## Train RNN - biLSTM Model

In [7]:
from utils.train import train_rnn_model_with_parameters

Hieuristic search with Optuna

In [8]:
import optuna
from utils.train import train_rnn_model_with_parameters

SEARCH_SPACE = {
    "batch_size": [2048],
    "learning_rate": [1e-1, 1e-2, 1e-3, 1e-4],
    "optimizer_name": ["Adam", "Adagrad",],
    # RNN Model Parameters
    "hidden_dim": [256, 128, 64, 32],
    "num_layers": [1, 2, 4],
    "sentence_representation_type": ["last", "average", "max"],
}

def objective(trial):
    hidden_dim = trial.suggest_categorical("hidden_dim", SEARCH_SPACE["hidden_dim"])
    num_layers = trial.suggest_int("num_layers", min(SEARCH_SPACE["num_layers"]), max(SEARCH_SPACE["num_layers"]))
    optimizer_name = trial.suggest_categorical("optimizer_name", SEARCH_SPACE["optimizer_name"])
    batch_size = trial.suggest_categorical("batch_size", SEARCH_SPACE["batch_size"])
    learning_rate = trial.suggest_categorical("learning_rate", SEARCH_SPACE["learning_rate"])
    sentence_representation_type = trial.suggest_categorical("sentence_representation_type", SEARCH_SPACE["sentence_representation_type"])
    
    log_message = f"---------- batch_size_{batch_size}; lr_{learning_rate}; optimizer_{optimizer_name}; hidden_dim_{hidden_dim}; num_layers_{num_layers}; sentence_representation_{sentence_representation_type} ----------"
    print(log_message)

    val_acc = train_rnn_model_with_parameters(
        embedding_matrix=embedding_matrix,
        train_dataset=train_dataset,
        val_dataset=val_dataset,
        batch_size=batch_size,
        learning_rate=learning_rate,
        optimizer_name=optimizer_name,
        hidden_dim=hidden_dim,
        num_layers=num_layers,
        sentence_representation_type=sentence_representation_type,
        show_progress=True,
        log_dir="biLSTM-oov/test",
        rnn_type="LSTM",
        bidirectional=True,
        freeze_embedding=False
    )
    
    return val_acc

# Set up the Optuna study
study = optuna.create_study(direction="maximize") 
study.optimize(objective, n_trials=10) 

best_params = study.best_params

[I 2024-11-08 13:57:56,055] A new study created in memory with name: no-name-b254b3cf-ac55-40ab-b247-24130b48cfb1
Seed set to 42
/Users/bern/anaconda3/envs/nlp_project_new/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'rnn_model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['rnn_model'])`.
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/bern/anaconda3/envs/nlp_project_new/lib/python3.11/site-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.

  | Name   | Type               | Params | Mode 
------------------------------------------------------
0 | model  | RNN                | 5.0 M  | train
1 | metric | MulticlassAccuracy | 0      | train
------------------------------------------------------
5.

---------- batch_size_2048; lr_0.001; optimizer_Adagrad; hidden_dim_32; num_layers_3; sentence_representation_last ----------
Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/bern/anaconda3/envs/nlp_project_new/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:419: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


/Users/bern/anaconda3/envs/nlp_project_new/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:419: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Epoch 15: 100%|██████████| 5/5 [01:01<00:00,  0.08it/s, v_num=0, train_loss=0.346, train_acc=0.874, val_loss=0.545, val_acc=0.750]


...

[I 2024-11-08 15:30:40,119] Trial 7 finished with value: 0.6018025875091553 and parameters: {'hidden_dim': 32, 'num_layers': 4, 'optimizer_name': 'Adam', 'batch_size': 2048, 'learning_rate': 0.0001, 'sentence_representation_type': 'average'}. Best is trial 1 with value: 0.7769343256950378.
Seed set to 42
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name   | Type               | Params | Mode 
------------------------------------------------------
0 | model  | RNN                | 6.2 M  | train
1 | metric | MulticlassAccuracy | 0      | train
------------------------------------------------------
6.2 M     Trainable params
0         Non-trainable params
6.2 M     Total params
24.699    Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode


---------- batch_size_2048; lr_0.01; optimizer_Adagrad; hidden_dim_256; num_layers_1; sentence_representation_last ----------
Epoch 8: 100%|██████████| 5/5 [00:41<00:00,  0.12it/s, v_num=0, train_loss=0.147, train_acc=0.924, val_loss=0.694, val_acc=0.738]


[I 2024-11-08 15:37:21,499] Trial 8 finished with value: 0.5881122946739197 and parameters: {'hidden_dim': 256, 'num_layers': 1, 'optimizer_name': 'Adagrad', 'batch_size': 2048, 'learning_rate': 0.01, 'sentence_representation_type': 'last'}. Best is trial 1 with value: 0.7769343256950378.
Seed set to 42
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name   | Type               | Params | Mode 
------------------------------------------------------
0 | model  | RNN                | 9.3 M  | train
1 | metric | MulticlassAccuracy | 0      | train
------------------------------------------------------
9.3 M     Trainable params
0         Non-trainable params
9.3 M     Total params
37.315    Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode


---------- batch_size_2048; lr_0.01; optimizer_Adagrad; hidden_dim_256; num_layers_3; sentence_representation_max ----------
Epoch 5: 100%|██████████| 5/5 [01:07<00:00,  0.07it/s, v_num=0, train_loss=0.693, train_acc=0.500, val_loss=0.693, val_acc=0.500]


[I 2024-11-08 15:44:44,911] Trial 9 finished with value: 0.6928321123123169 and parameters: {'hidden_dim': 256, 'num_layers': 3, 'optimizer_name': 'Adagrad', 'batch_size': 2048, 'learning_rate': 0.01, 'sentence_representation_type': 'max'}. Best is trial 1 with value: 0.7769343256950378.


In [9]:
best_params

{'hidden_dim': 256,
 'num_layers': 2,
 'optimizer_name': 'Adagrad',
 'batch_size': 2048,
 'learning_rate': 0.1,
 'sentence_representation_type': 'last'}

## Model Configurations Comparison

In [4]:
from utils.analytics import load_tensorboard_logs

train_results_df = load_tensorboard_logs(log_dir="tb_logs/bilstm-oov")
train_results_df = train_results_df.sort_values(
    by=["val_acc"], ascending=False
).reset_index(drop=True)

print("Results for biLSTM with OOV method")
train_results_df.head(10)


DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in match_rnn_log
DATA in ma

,val_acc,batch_size,hidden_dim,learning_rate,optimizer_name,train_loss,train_acc,num_layers,sentence_representation_type,freeze,epoch,val_loss,filename
0,0.791745,2048,128,0.0100,Adagrad,0.092932,0.961358,1,average,False,8.0,0.524783,events.out.tfevents.1731018611.Bernices-MacBoo...
1,0.786116,2048,64,0.0010,Adam,0.117063,0.967680,1,max,False,10.0,0.486323,events.out.tfevents.1731013380.Bernices-MacBoo...
2,0.779550,2048,64,0.0010,Adagrad,0.316925,0.881039,1,max,False,21.0,0.487491,events.out.tfevents.1731017839.Bernices-MacBoo...
3,0.778612,2048,256,0.0010,Adagrad,0.298538,0.867635,1,max,False,14.0,0.489787,events.out.tfevents.1730998423.Bernices-MacBoo...
4,0.773921,2048,128,0.0100,Adagrad,0.241678,0.920623,3,max,False,6.0,0.517354,events.out.tfevents.1730990217.Bernices-MacBoo...
5,0.772983,2048,64,0.0010,Adam,0.100994,0.966198,1,last,False,10.0,0.519525,events.out.tfevents.1730994098.Bernices-MacBoo...
6,0.772045,2048,32,0.0100,Adagrad,0.205683,0.936922,2,average,False,6.0,0.558855,events.out.tfevents.1730993100.Bernices-MacBoo...
7,0.769231,2048,32,0.0001,Adam,0.212386,0.942633,1,max,False,59.0,0.476411,events.out.tfevents.1730994491.Bernices-MacBoo...
8,0.768293,2048,128,0.0100,Adagrad,0.219571,0.932048,4,max,False,6.0,0.525361,events.out.tfevents.1730992028.Bernices-MacBoo...
9,0.765478,2048,32,0.0010,Adam,0.174375,0.931984,2,last,False,8.0,0.562326,events.out.tfevents.1731008580.Bernices-MacBoo...


# (a) Final Configuration of best model

In [21]:
best_rnn_model_configuration = train_results_df.head(1)

print("Best RNN model (with OOV)")
best_rnn_model_configuration

Best RNN model (with OOV)


,val_acc,batch_size,hidden_dim,learning_rate,optimizer_name,train_loss,train_acc,num_layers,sentence_representation_type,freeze,epoch,val_loss,filename
0,0.791745,2048,128,0.01,Adagrad,0.092932,0.961358,1,average,False,8.0,0.524783,events.out.tfevents.1731018611.Bernices-MacBoo...


In [22]:
from pathlib import Path
from models.RNN import RNNClassifier

best_rnn_model_filename = best_rnn_model_configuration["filename"].item()
matched_files = list(Path().rglob(best_rnn_model_filename))

if not matched_files:
    print("Model checkpoint not found!")
else:
    checkpoint_dir = matched_files[0].parent / "checkpoints"
    checkpoint_files = (
        list(checkpoint_dir.glob("*.ckpt")) if checkpoint_dir.exists() else []
    )

    if not checkpoint_files:
        print("No checkpoint files found in the checkpoint directory!")
    else:
        best_checkpoint = checkpoint_files[0] 
        print("best checkpoint: ", best_checkpoint)
        best_rnn_model = RNNClassifier.load_from_checkpoint(best_checkpoint)
        print(best_rnn_model)

best checkpoint:  tb_logs/biLSTM-oov/test/batch_size_2048-lr_0.01-optimizer_Adagrad-hidden_dim_128-num_layers_1-sr_type_average-freeze_False-rnn_type_LSTM-bidirectional_True/version_0/checkpoints/epoch=5-step=30.ckpt
RNNClassifier(
  (model): RNN(
    (embedding): Embedding(16334, 300)
    (rnn): LSTM(300, 128, batch_first=True, bidirectional=True)
    (fc): Linear(in_features=256, out_features=128, bias=True)
    (relu): ReLU()
    (fc2): Linear(in_features=128, out_features=2, bias=True)
  )
  (metric): MulticlassAccuracy()
)


/Users/bern/anaconda3/envs/nlp_project_new/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'rnn_model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['rnn_model'])`.


# (b) Accuracy on Testset

In [23]:
import lightning as L
from torch.utils.data import DataLoader

test_dataloader = DataLoader(test_dataset, shuffle=True)

trainer = L.Trainer(accelerator="cpu")
trainer.test(best_rnn_model, test_dataloader)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


/Users/bern/anaconda3/envs/nlp_project_new/lib/python3.11/site-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/Users/bern/anaconda3/envs/nlp_project_new/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:475: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/Users/bern/anaconda3/envs/nlp_project_new/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 1066/1066 [00:02<00:00, 360.58it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.7833020687103271
        test_loss           0.6383655071258545
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.6383655071258545, 'test_acc': 0.7833020687103271}]